# Exercice 1: Reccurent neuronal network: IMBD sentiment classification

## Our goal here will be to be able to predict if a comment is positive or negative based only on the text comment

### First, we importe all the needed libraries

In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import keras
import re
import tarfile
import csv
import io
import os
from collections import defaultdict
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer

### We preview the dataset

In [2]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [3]:
english_stops = set(stopwords.words('english'))
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

### Load and Clean Dataset

since we do not 

### Encode Sentiments

In [4]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


### Split dataset

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(y_train, '\n')
print('Test Set')
print(x_test, '\n')
print(y_test)

Train Set
21437    [although, movie, slow, dream, like, almost, m...
5001     [this, film, typical, hollywood, fare, though,...
47386    [sarah, silverman, dangerous, bitch, she, beau...
45303    [an, amazing, film, i, seen, i, already, want,...
3728     [if, film, noir, noir, must, french, glacially...
                               ...                        
24886    [this, movie, advertised, radio, television, m...
19639    [found, one, video, store, rented, it, one, qu...
48731    [i, know, neighborhoods, folks, wrote, rave, r...
3921     [i, think, lion, king, one, best, sequels, eve...
42828    [for, love, god, please, see, movie, its, wast...
Name: review, Length: 40000, dtype: object 

21437    0
5001     1
47386    1
45303    1
3728     0
        ..
24886    0
19639    1
48731    0
3921     1
42828    0
Name: sentiment, Length: 40000, dtype: int64 

Test Set
43216    [ocean, twelve, plain, stupid, bad, nothing, c...
20978    [broadway, film, actor, turned, director, john...
4

In [6]:
#function for getting the maximum review length
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

### Tokenize and Pad/Truncate Reviews

In [7]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[  165     3   466 ...     0     0     0]
 [    8     4   714 ...     0     0     0]
 [ 2551  8465  1638 ...     0     0     0]
 ...
 [    1    48 15907 ... 21511     0     0]
 [    1    31  2829 ...     0     0     0]
 [  204    40   404 ...     0     0     0]] 

Encoded X Test
 [[2912 4115  943 ...    0    0    0]
 [2356    4  180 ...   28 1030 3091]
 [   8   99  867 ...    0    0    0]
 ...
 [1234    3   55 ...    0    0    0]
 [   8 3847   17 ...    0    0    0]
 [   8    3 1018 ...    0    0    0]] 

Maximum review length:  130


### Build Architecture/Model

In [8]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 130, 32)           2956864   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 2,981,761
Trainable params: 2,981,761
Non-trainable params: 0
_________________________________________________________________
None


### Training

In [9]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)
model.fit(x_train, y_train, batch_size = 128, epochs = 5, callbacks=[checkpoint])

Epoch 1/5
313/313 [==============================] - 25s 75ms/step - loss: 0.6001 - accuracy: 0.6186

Epoch 00001: accuracy improved from -inf to 0.74630, saving model to models\LSTM.h5
Epoch 2/5
313/313 [==============================] - 24s 76ms/step - loss: 0.2101 - accuracy: 0.9264

Epoch 00002: accuracy improved from 0.74630 to 0.92385, saving model to models\LSTM.h5
Epoch 3/5
313/313 [==============================] - 24s 75ms/step - loss: 0.1164 - accuracy: 0.9632

Epoch 00003: accuracy improved from 0.92385 to 0.95985, saving model to models\LSTM.h5
Epoch 4/5
313/313 [==============================] - 23s 75ms/step - loss: 0.0755 - accuracy: 0.9803

Epoch 00004: accuracy improved from 0.95985 to 0.97723, saving model to models\LSTM.h5
Epoch 5/5
313/313 [==============================] - 24s 76ms/step - loss: 0.0510 - accuracy: 0.9869

Epoch 00005: accuracy improved from 0.97723 to 0.98480, saving model to models\LSTM.h5


### Testing

In [10]:
y_pred = model.predict_classes(x_test, batch_size = 128)

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

C:\Users\Megaport\anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Correct Prediction: 8672
Wrong Prediction: 1328
Accuracy: 86.72


### Load Saved Model

In [ ]:
loaded_model = load_model('models/LSTM.h5')
review = str(input('Movie Review: '))

In [ ]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

In [ ]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

In [ ]:
result = loaded_model.predict(tokenize_words)
print(result)

### Predict the sentiment of short textual comments about movies

In [ ]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

# Exercice 2: Text Classification: the Ohsumed dataset

## For this second exercice we are going to realize a text classifier using deep neural networks

### To beginning, we are going to use the dataset that contains 20 000 articles

#### To begin, we are just going to understand how the data works

In [2]:
## Parse your data
def get_info(path: str):
    data = list(os.walk(path))[1:]
    files = []
    categories = []
    textes = []
    for d in data:
        folder_name = d[0]
        for file in d[2]:
            s = folder_name.split('/')[-1].split("\\")[1]
            categories.append(s) # the tags of our model
            files.append((folder_name.split('/')[-1], os.path.join(folder_name, file)))

    d = defaultdict(int)
    texts = defaultdict(list)
    for (cate, file) in files:
        with open(file, 'r') as outfile:
            text = outfile.read()
            texts[cate].append(text)
            textes.append(text)
            words = text_to_word_sequence(text)
            for word in words:
                d[word] += 1
    data = pd.DataFrame(data={'Categories' : categories, 'Textes' : textes}) # a dataFrame with the categorie corresponding to its text description
    words = sorted(d.items(), key=lambda x: x[1], reverse=True)
    return (data, words)

In [3]:
# First we import the dataset of 20 000
cwd = os.getcwd()
print('Current working directory :', cwd)
data_dir_training = cwd + "/ohsumed-first-20000-docs/training"
data_dir_test = cwd + "/ohsumed-first-20000-docs/test"
data_train, words = get_info(data_dir_training)
data_test, words_test = get_info(data_dir_test)

Current working directory : C:\Users\Megaport\OneDrive\Bureau\IA\TP_1_et_2_IA


In [4]:
data_train

,Categories,Textes
0,C01,Augmentation mentoplasty using Mersilene mesh....
1,C01,Multiple intracranial mucoceles associated wit...
2,C01,Replacement of an aortic valve cusp after neon...
3,C01,The value of indium 111 leukocyte scanning in ...
4,C01,Febrile infants less than eight weeks old. Pre...
...,...,...
10428,C23,Afferent nipple valve malfunction caused by an...
10429,C23,Extracorporeal shock wave lithotripsy in combi...
10430,C23,Tamm-Horsfall autoantibodies in interstitial c...
10431,C23,Results of contemporary radical cystectomy for...


In [5]:
data_test

,Categories,Textes
0,C01,Haemophilus influenzae meningitis with prolong...
1,C01,Mucosal intussusception to avoid ascending cho...
2,C01,Gastrointestinal function and structure in HIV...
3,C01,Epidemiology in bone and joint infection.\n Th...
4,C01,The diabetic foot. Soft tissue and bone infect...
...,...,...
12728,C23,Interruption of professional and home activity...
12729,C23,Effect of thiopental on neurologic outcome fol...
12730,C23,Recovery profile after desflurane-nitrous oxid...
12731,C23,Postoperative myocardial ischemia in patients ...


In [14]:
valeurMax = 0
for i in range(len(words)):
    valeurMax += words[i][1]
listeMots = []
listePourcentage = []
for i in range(len(words)):
    listeMots.append(words[i][0])
    listePourcentage.append(((words[i][1]/valeurMax) * 100))
d = {'words': listeMots, 'percentage': listePourcentage}
df = pd.DataFrame(data=d)
listeMots = []
listePourcentage = []
for i in range(len(words_test)):
    listeMots.append(words_test[i][0])
    listePourcentage.append(((words_test[i][1]/valeurMax) * 100))
d = {'words': listeMots, 'percentage': listePourcentage}
dt = pd.DataFrame(data=d)
print(df)
print(dt)
print(len(data_train))
print(len(data_test))

                        words  percentage
0                         the    4.573823
1                          of    4.545638
2                         and    3.080502
3                          in    2.964911
4                          to    1.660441
...                       ...         ...
30852                   dtssp    0.000054
30853                opossums    0.000054
30854               131iodine    0.000054
30855  intraperitonealization    0.000054
30856                    ball    0.000054

[30857 rows x 2 columns]
                  words  percentage
0                    of    5.738498
1                   the    5.660075
2                   and    3.855590
3                    in    3.711868
4                  with    2.071651
...                 ...         ...
34365           greeted    0.000054
34366          tempered    0.000054
34367              earn    0.000054
34368  nonmitochondrial    0.000054
34369           saponin    0.000054

[34370 rows x 2 columns]
10433
12733


#### We are now going to prepocess our data

In [8]:
english_stops = set(stopwords.words('english'))

In [20]:
def load_dataset(data):
    x_data = data['Textes']      # Text/Input
    y_data = data['Categories']      # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('C01', 1)
    y_data = y_data.replace('C02', 2)
    y_data = y_data.replace('C03', 3)
    y_data = y_data.replace('C04', 4)
    y_data = y_data.replace('C05', 5)
    y_data = y_data.replace('C06', 6)
    y_data = y_data.replace('C07', 7)
    y_data = y_data.replace('C08', 8)
    y_data = y_data.replace('C09', 9)
    y_data = y_data.replace('C10', 10)
    y_data = y_data.replace('C11', 11)
    y_data = y_data.replace('C12', 12)
    y_data = y_data.replace('C13', 13)
    y_data = y_data.replace('C14', 14)
    y_data = y_data.replace('C15', 15)
    y_data = y_data.replace('C16', 16)
    y_data = y_data.replace('C17', 17)
    y_data = y_data.replace('C18', 18)
    y_data = y_data.replace('C19', 19)
    y_data = y_data.replace('C20', 20)
    y_data = y_data.replace('C21', 21)
    y_data = y_data.replace('C22', 22)
    y_data = y_data.replace('C23', 23)

    return x_data, y_data

x_train, y_train = load_dataset(data_train)
x_test, y_test = load_dataset(data_test)

print('Data trains')
print(x_train, '\n')
print(y_train, '\n')
print('Data test')
print(x_test, '\n')
print(y_test)

Data trains
0        [augmentation, mentoplasty, using, mersilene, ...
1        [multiple, intracranial, mucoceles, associated...
2        [replacement, aortic, valve, cusp, neonatal, e...
3        [the, value, indium, leukocyte, scanning, eval...
4        [febrile, infants, less, eight, weeks, old, pr...
                               ...                        
10428    [afferent, nipple, valve, malfunction, caused,...
10429    [extracorporeal, shock, wave, lithotripsy, com...
10430    [tamm, horsfall, autoantibodies, interstitial,...
10431    [results, contemporary, radical, cystectomy, i...
10432    [total, bladder, replacement, using, detubular...
Name: Textes, Length: 10433, dtype: object 

0         1
1         1
2         1
3         1
4         1
         ..
10428    23
10429    23
10430    23
10431    23
10432    23
Name: Categories, Length: 10433, dtype: int64 

Data test
0        [haemophilus, influenzae, meningitis, prolonge...
1        [mucosal, intussusception, avoid, as

In [ ]:
y_train, y_test

#### We are going to tokenise and Pad/Truncate Reviews

#### And now, we can create our own model

### Now that we achieved it on a small dataset, we are going to do it on a more wide dataset of 50 000 articles